# PADIWA Rate Scan for PMTSpec

In [ ]:
%matplotlib notebook
#%matplotlib qt
#!/usr/bin/env python
# coding: utf-8

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates
import copy 

import scipy
import scipy.stats
import scipy.signal
import scipy.ndimage
from scipy.interpolate import interp1d

import tqdm
import tqdm.notebook

import cv2

import os

from inspect import getfullargspec

import tables

import strawb

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
# get DB and update
db = strawb.SyncDBHandler()
db.load_onc_db_update(output=True, save_db=True)

In [ ]:
f_s = ['TUMPMTSPECTROMETER001_20220310T030107.064Z-SDAQ-PMTSPEC.hdf5',
       'TUMPMTSPECTROMETER001_20220317T130834.712Z-SDAQ-PMTSPEC.hdf5',
       'TUMPMTSPECTROMETER001_20220317T144950.271Z-SDAQ-PMTSPEC.hdf5',
       'TUMPMTSPECTROMETER001_20220318T152927.297Z-SDAQ-PMTSPEC.hdf5',
       'TUMPMTSPECTROMETER001_20220321T110607.536Z-SDAQ-PMTSPEC.hdf5']

print('check if files are available or download missing files')
db.get_files_from_names(f_s)

In [ ]:
pmt_all = []
print('Load the PMTSpec Instances')
for i in tqdm.notebook.tqdm(f_s):
    pmt = strawb.sensors.PMTSpec(i)
    pmt.pmt_meta_data.add_colors()
    
    pmt_all.append(pmt)

In [ ]:
print('Check if all scans get extracted')
for pmt_i in tqdm.notebook.tqdm(pmt_all):
    pmt_i.rate_scan.scan_off
    pmt_i.rate_scan.scan_on

In [ ]:
pmt_i.file_handler.get_pandas_padiwa()

# test the unique_steps function

In [ ]:
states = [np.array([-1,0,0,0,1,1,1,1,0,0,2,2,2,3,4,4,5,5]),
          np.array([-1,-1,0,0,0,1,1,1,1,0,0,2,2,2,3,4,4,5]),
#           np.array([-1,-1,0,0,0,1,1,1,1,0,0,2,2,2,3,4,4,5,5]),
         ]


for state in states:
    t = np.arange(len(state))
    
    t_steps, state_steps = strawb.tools.unique_steps(t, state)
    
    plt.figure()
    plt.plot(t, state, 'o:', label='raw', ms=7, alpha=1)
    plt.plot(t_steps.T, state_steps.T, 'o-', ms=10, label='t_steps', alpha=.5)
    # plt.plot(tt.T, state[mm].T, 'o--')-
    # tt, state[mm].T, m, mm
    plt.legend()
    
t_steps, state_steps

# Store Scan parameters and calculate the rate per threshold

In [ ]:
def plot_time(pmt):
    fig, ax = plt.subplots(nrows=3, figsize=(10,10), squeeze=False, sharex=True)
    ax = ax.flatten()

    scans = [pmt.rate_scan.scan_off, pmt.rate_scan.scan_on]
    
    for i, scan_i in enumerate(scans):
        ax[0].plot(scan_i.time_padiwa,
                   scan_i.steps_padiwa,
                   'o-', label=scan_i.name)

        for ch in pmt.pmt_meta_data.channel_meta_array:
            if i == 0:
                label=ch['label']
            else:
                label=None
            
            ax[1].plot(scan_i.time_interpolated,
                       scan_i.rate_interpolated[ch['index']], 
                       'o-',
                       color=ch['color'], label=label, 
                       alpha=.5)

            ax[2].plot(scan_i.time_interpolated, 
                       scan_i.active_ratio[ch['index']], 
                       'o-',
                       color=ch['color'], label=label, 
                       alpha=.5)

    ax[0].set_ylabel('PADIWA Offsets')
    ax[1].set_ylabel('avg. Rate per Step [Hz]')
    ax[2].set_ylabel('ratio of active counter reads')
    fig.supxlabel('Date')
    for ax_i in ax:
        ax_i.grid()
        ax_i.legend(ncol=2)

    fig.suptitle(os.path.basename(pmt.file_handler.file_name))
    plt.tight_layout()
    plt.show()
    

for pmt_i in pmt_all:
    plot_time(pmt_i)

# Plot Threshold vs. rate (interpolated)

In [ ]:
def threshold_vs_rate(pmt):
    fig, ax = plt.subplots(nrows=12, sharex=True, sharey=True, constrained_layout=True, figsize=(10,15))

    ax = ax.flatten()
    
    scans = [pmt.rate_scan.scan_off, pmt.rate_scan.scan_on]

    i_ax = 0
    for ch in pmt.pmt_meta_data.channel_meta_array:
        # print(ch)
        threshold_0 = pmt.file_handler.__getattribute__(f"padiwa_th{ch['padiwa']}")[0]
        offset_0 = pmt.file_handler.padiwa_offset[0]
            
#         print(scan_threshold, threshold_0)
        for scan_i in scans:
            # Get threshold: Threshold start - offset
            thresholds = scan_i.scan_threshold_dict[ch['padiwa']] - scan_i.steps_padiwa[:-1]
            ax[i_ax].plot(thresholds[::2], 
                          scan_i.rate_interpolated[ch['index'], ::2], '-',
                          label=f"Rate {scan_i.name}")

            ax[i_ax].plot(thresholds[::2], 
                          scan_i.active_ratio[ch['index'], ::2], '--',
                          label=f"Active count {scan_i.name}")

        ax[i_ax].legend(loc='center left', 
                        bbox_to_anchor=(1.0, 0.5), 
                        title=f"Channel: {ch['label']}")

        threshold = pmt.file_handler.__getattribute__(f"padiwa_th{ch['padiwa']}")[0]
        offset = pmt.file_handler.padiwa_offset[0]

        # the x coords of this transformation are data, and the
        # y coord are axes
        trans = ax[i_ax].get_xaxis_transform()
        ax[i_ax].axvline(x=threshold_0-offset_0, label='threshold', color='k')
        ax[i_ax].text(threshold_0-offset_0-100, .90, 'current threshold', 
                      #rotation=90, 
                      fontsize=12,
                      horizontalalignment='right',
                      verticalalignment='top',
                      transform=trans,
                      bbox=dict(boxstyle="round",
                                alpha=.75,
                                ec='gray',
                                fc=(1., 1., 1.),),
                     )
        i_ax += 1

    for ax_i in ax:
        ax_i.set_yscale('log')
    #     ax_i.legend(loc=1, ncol=2)
    #     ax_i.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        ax_i.grid()

    fig.supxlabel('PADIWA Threshold')
    fig.supylabel('Rate [Hz] or Counts')    
    fig.suptitle(os.path.basename(pmt.file_handler.file_name))
    plt.show()
    
for pmt_i in pmt_all:
    threshold_vs_rate(pmt_i)